<a href="https://colab.research.google.com/github/aryelmedeiros/eng-dados-Spark-Kafka-Airflow/blob/main/apps/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Utilizando o Spark no Google Colab

Para facilitar o desenvolvimento de nosso projeto neste curso vamos utilizar o Google Colab como ferramenta e para configurar o PySpark basta executar os comandos abaixo na própria célula do seu *notebook*.

In [ ]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #instalação do java8
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz #versão do spark, download e descompactação
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
# variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

# Carregamento de Dados
---

## Acessando o [Spark UI](https://spark.apache.org/docs/3.1.2/web-ui.html) (Google Colab)

In [ ]:
#para acessar o spark UI, comenta o trecho acima e encerre a sessão
# diferença é a porta
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

[Site ngrok](https://ngrok.com)

In [ ]:
# caminho seguro na nuvem
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# criando o caminho que vai rodar na porta 4050, rodando em segundo plano
# criar conta no https://dashboard.ngrok.com/login
get_ipython().system_raw('./ngrok authtoken 2ZWGhV11RMRHj5yIjZ7ubKIGhXC_5LiDCe8Frh3rZZHcJ5Br5')
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
# acessando a porta 4050, com uma url pública
!curl -s http://localhost:4040/api/tunnels


{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://6c30-35-189-164-170.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://6c30-35-189-164-170.ngrok-free.app","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [ ]:
spark

In [ ]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError

In [ ]:
!curl -sSOL https://downloads.apache.org/kafka/3.4.1/kafka_2.13-3.4.1.tgz
!tar -xzf kafka_2.13-3.4.1.tgz

In [ ]:
!./kafka_2.13-3.4.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.4.1/config/zookeeper.properties
!./kafka_2.13-3.4.1/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.4.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!ps -ef | grep kafka

root       49810    1837  0 08:18 ?        00:00:00 /bin/bash -c ps -ef | grep kafka
root       49812   49810  0 08:18 ?        00:00:00 grep kafka


In [ ]:
!./content/kafka_2.13-3.4.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic topico1

/bin/bash: line 1: ./content/kafka_2.13-3.4.1/bin/kafka-topics.sh: No such file or directory


In [ ]:
!wget https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/estabelecimentos.zip

--2023-12-14 08:07:20--  https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/estabelecimentos.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 54.231.128.129, 16.182.37.249, 3.5.29.169, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|54.231.128.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 310117201 (296M) [application/zip]
Saving to: ‘estabelecimentos.zip’

estabelecimentos.zi 100%[===================>] 295.75M  15.6MB/s    in 20s     

2023-12-14 08:07:41 (14.6 MB/s) - ‘estabelecimentos.zip’ saved [310117201/310117201]



In [ ]:
df = spark.read.csv("/content/cars.csv", header=True, inferSchema=True)
df.show()

+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|manufacturer_name|model_name|transmission| color|odometer_value|year_produced|engine_fuel|engine_has_gas|engine_type|engine_capacity|body_type|has_warranty|state|drivetrain|price_usd|is_exchangeable|location_region|number_of_photos|up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|duration_listed|
+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+----

In [ ]:
df.printSchema()

root
 |-- manufacturer_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: integer (nullable = true)
 |-- year_produced: integer (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: boolean (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: double (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: boolean (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: double (nullable = true)
 |-- is_exchangeable: boolean (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: integer (nullable = true)
 |-- up_counter: integer (nullable = true)
 |-- feature_0: boolean (nullable = true)
 |-- feature_1: boolean (nullable = true)
 |-- feature_2: boolean (nullable = true)
 |-- feature_3: boolean (nullab

In [ ]:
modelo_predominante = df.groupBy(['model_name']).count()
modelo_predominante = modelo_predominante.sort("count", ascending=False)
modelo_predominante.show()

+----------+-----+
|model_name|count|
+----------+-----+
|    Passat| 1423|
|     Astra|  751|
|      Golf|  707|
|        A6|  687|
|    Mondeo|  637|
|    Vectra|  565|
|    Laguna|  548|
|        A4|  505|
|       406|  415|
|     Omega|  387|
|     Focus|  380|
|    Megane|  377|
|       100|  371|
|   Octavia|  367|
|    Zafira|  356|
|    Xantia|  355|
|       525|  354|
|   Primera|  346|
|    Scenic|  323|
|       626|  321|
+----------+-----+
only showing top 20 rows



In [ ]:
modelo_predominante.write.mode("overwrite").options(
    header='True', delimiter=',').csv("/content/analises/modelo_predominante")